## Country Club SQL Case Study

In [1]:
# Copy and paste the LocalSQLConnection.py script into an empty Jupyter notebook, and run it. 
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


### Questions 1 - 9 : Completed on PHPMYADMIN

Solutions from 1 to 9 are provided here for easy reference in the mark-down format only:

**Question 1:** Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.

**ANSWER: 1** 

SELECT *
FROM Facilities
WHERE membercost =0
LIMIT 0 , 30

**Question 2:** How many facilities do not charge a fee to members?

**ANSWER: 2** 

SELECT COUNT(*)
FROM Facilities
WHERE membercost=0;

**Question 3:** Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

**ANSWER: 3** 

SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost>0 AND membercost < monthlymaintenance * 0.2;

**Question 4:** Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.

**ANSWER: 4** 

SELECT *
FROM Facilities
WHERE facid IN (1,5);

**Question 5:** Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.  

**ANSWER: 5** 

SELECT name, 
CASE WHEN monthlymaintenance>100 THEN 'Expensive'
WHEN monthlymaintenance<100 THEN 'Cheap' END AS facility_label
FROM Facilities;


**Question 6:** You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

**ANSWER: 6** 

SELECT firstname, surname, joindate
FROM Members
WHERE joindate=(SELECT MAX(joindate) FROM Members);

**Question 7:** Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

**ANSWER: 7** 

SELECT DISTINCT name, CONCAT(firstname, surname) AS member
FROM Facilities
INNER JOIN Bookings
ON Facilities.facid = Bookings.facid
INNER JOIN Members
ON Bookings.memid = Members.memid
WHERE name LIKE 'Tennis Court%'
ORDER BY CONCAT(firstname, surname);

**Question 8:** Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

**ANSWER: 8** 

SELECT name,CONCAT(firstname, surname) AS member_name, membercost, guestcost, membercost + guestcost AS total_cost, starttime
FROM Facilities
INNER JOIN Bookings
ON Facilities.facid = Bookings.facid
INNER JOIN Members
ON Members.memid = Bookings.memid
WHERE starttime BETWEEN '2012-09-14' AND '2012-09-15'
HAVING total_cost>30
ORDER BY total_cost DESC;

**Question 9:** This time, produce the same result as in Q8, but using a subquery.

**ANSWER: 9** 

SELECT sub.name, sub.member_name, sub.total_cost
FROM (SELECT name,CONCAT(firstname, surname) AS member_name, membercost, guestcost, membercost + guestcost AS total_cost, starttime
FROM Facilities
INNER JOIN Bookings
ON Facilities.facid = Bookings.facid
INNER JOIN Members
ON Members.memid = Bookings.memid
ORDER BY total_cost DESC) AS sub
WHERE starttime BETWEEN '2012-09-14' AND '2012-09-15'
AND total_cost>30;

**Question 10: Use Jupyter Notebook for solution** 

Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [2]:
import sqlite3  
from sqlite3 import Error 
import pandas as pd

In [3]:
conn = sqlite3.connect("./sqlite_db_pythonsqlite.db")

In [4]:
cur = conn.cursor()

In [5]:
q = """
SELECT name as Facility_Name, SUM(cost) AS Revenue FROM
(SELECT Facilities.facid, Facilities.name,
CASE WHEN Bookings.memid = CAST(0 AS integer) 
THEN CAST(Facilities.guestcost AS integer) * CAST(slots AS integer)
ELSE CAST(Facilities.membercost AS integer) * CAST(slots AS integer)
END as 'cost'
FROM Bookings
INNER JOIN Facilities ON Bookings.facid = Facilities.facid)
GROUP BY facid
HAVING Revenue < 1000
ORDER BY Revenue DESC
"""

In [6]:
pd.read_sql(q, conn)

,Facility_Name,Revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


**Question 11: Use Jupyter Notebook for solution** 

Produce a report of members and who recommended them in alphabetic surname,firstname order.

In [7]:
q = """
SELECT (surname || ' '|| firstname) as full_name, 
            (SELECT (firstname || ' ' || surname) as recommended_by FROM members AS inner
            WHERE inner.memid = Members.recommendedby) as recommendedby
    FROM Members
    ORDER BY full_name
"""

In [8]:
pd.read_sql(q, conn)

,full_name,recommendedby
0,Bader Florence,Ponder Stibbons
1,Baker Anne,Ponder Stibbons
2,Baker Timothy,Jemima Farrell
3,Boothe Tim,Tim Rownam
4,Butters Gerald,Darren Smith
5,Coplin Joan,Timothy Baker
6,Crumpet Erica,Tracy Smith
7,Dare Nancy,Janice Joplette
8,Farrell David,None
9,Farrell Jemima,None


**Question 12: Use Jupyter Notebook for solution** 

Find the facilities with their usage by member, but not guests.

In [9]:
q = '''
SELECT name 
FROM Bookings
INNER JOIN Facilities
ON Bookings.facid = Facilities.facid
WHERE memid !=0
GROUP BY name
'''

In [10]:
pd.read_sql(q, conn)

,name
0,Badminton Court
1,Massage Room 1
2,Massage Room 2
3,Pool Table
4,Snooker Table
5,Squash Court
6,Table Tennis
7,Tennis Court 1
8,Tennis Court 2


**Question 13: Use Jupyter Notebook for solution** 

Find the facilities usage by month, but not guests.

In [13]:
q = '''
SELECT strftime('%m', starttime) AS Month,Facilities.name as Facility_name, COUNT(*) AS times_used FROM Bookings
    INNER JOIN Facilities on Facilities.facid = Bookings.facid
    GROUP BY Month, name
'''

In [14]:
pd.read_sql(q, conn)

,Month,Facility_name,times_used
0,07,Badminton Court,56
1,07,Massage Room 1,123
2,07,Massage Room 2,12
3,07,Pool Table,110
4,07,Snooker Table,75
5,07,Squash Court,75
6,07,Table Tennis,51
7,07,Tennis Court 1,88
8,07,Tennis Court 2,68
9,08,Badminton Court,146
